## T5

This notebook is adapted from HuggingFace's summarisation notebook. https://huggingface.co/transformers/notebooks.html

# 1. Import Modules
# 2. Load Data
# 3. Preprocess the Data
# 4. Finetune the Model
# 3. Make Predictions

## Import Modules

In [ ]:
%pip install datasets transformers rouge-score nltk
%pip install wandb
import wandb
%pip install optuna
%pip install tqdm
!pip install ray==0.8.7
!pip install ray[tune]
import tqdm
import datasets
import transformers
import nltk
nltk.download('punkt')
from datasets import load_dataset, load_metric
%pip install pickle5
import pickle5 as pickle
import os
import torch
import datasets
import random
import pandas as pd
from datasets import Dataset
from ray import tune
from transformers import AutoTokenizer
import nltk
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# define model to be used
model_checkpoint = "t5-small"

## Loading the dataset

In [ ]:
# open a file, where you stored the pickled data
DATA_PATH = "/content"
file1 = open(DATA_PATH+'/tac_train_dataset_nodups2.pickle', 'rb')
file2 = open(DATA_PATH+'/tac_valid_dataset_nodups2.pickle', 'rb')
file3 = open(DATA_PATH+'/tac_test_dataset_nodups2.pickle', 'rb')

# dump information to that file
tac_train = pickle.load(file1)
tac_valid = pickle.load(file2)
tac_test = pickle.load(file3)

metric = load_metric("rouge")

In [ ]:
# View the training set
tac_train.head()

,Summary,Original Text
456,this service gives no guarantee regarding quality,any details as to products including product descriptions benefits quality and performance are provided by the supplier or manufacturer of the product and are reproduced for the convenience of our users this information is not verified or substantiated by sparknotes and sparknotes is not responsible for any inaccuracies or omissions
505,users may opt out of binding arbitration within 30 days of signing the agreement,if you are a new user of our services you can choose to reject this agreement to arbitrate opt out by mailing us a written opt out notice opt out notice the opt out notice must be postmarked no later than 30 days after the date you accept the user agreement for the first time you must mail the opt out notice to ebay inc attn litigation department re opt out notice 583 west ebay way draper ut 84020
55,the service can remove screen or edit your content without prior notice and without a reason,brainly reserves the right to moderate the brainly services andto remove screen or edit your content from the brainly services at oursole discretion at any time and for any reason or for no reason with nonotice to you
682,the services will notify users if its account has been affected by data breaches,you must immediately notify e foundation of any unauthorized uses of your account or any other breaches of security
459,the court of law governing the terms is in location british virgin islands,this privacy policy and our privacy practices will be subject exclusively to the laws of the british virgin islands


In [ ]:
# Avoid "Expected Byte but got integer" error later
tac_train["Summary"] = tac_train["Summary"].astype(str)
tac_train["Original Text"] = tac_train["Original Text"].astype(str)

tac_valid["Summary"] = tac_valid["Summary"].astype(str)
tac_valid["Original Text"] = tac_valid["Original Text"].astype(str)

tac_test["Summary"] = tac_test["Summary"].astype(str)
tac_test["Original Text"] = tac_test["Original Text"].astype(str)

In [ ]:
# View metric
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each predictions
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_agregator: Return aggregates if this is set to True
Retu

## Preprocessing the data

In [ ]:
# Import the tokenizer for T5
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_p

In [ ]:
if model_checkpoint in ["t5-small"]:
    prefix = "summarize: "
else:
    prefix = ""

In [ ]:
max(tac_train.astype('str').applymap(lambda x: len(x)).max())

546

In [ ]:
max_input_length = 549
max_target_length = 200

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Original Text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["Summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
train_dataset = Dataset.from_pandas(tac_train)
val_dataset = Dataset.from_pandas(tac_valid)
test_dataset = Dataset.from_pandas(tac_test)

raw_datasets = datasets.DatasetDict({"train" : train_dataset, "validation" : val_dataset, "test" : test_dataset})

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['Summary', 'Original Text', '__index_level_0__'],
        num_rows: 607
    })
    validation: Dataset({
        features: ['Summary', 'Original Text', '__index_level_0__'],
        num_rows: 208
    })
    test: Dataset({
        features: ['Summary', 'Original Text', '__index_level_0__'],
        num_rows: 53
    })
})

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Fine-tuning the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, TrainingArguments

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
# define evaluation function

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
import torch
torch.cuda.empty_cache()

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [ ]:
# 1. Start a W&B run
wandb.init(project='t5summarizer', entity='belin')
run_name = wandb.run.name
wandb.config
wandb.log({'loss': 0.2, 'epoch': 1})
%env WANDB_LOG_MODEL=true

wandb: Currently logged in as: belin (use `wandb login --relogin` to force relogin)


env: WANDB_LOG_MODEL=true


In [ ]:
# Hyperparameter optimisation using Ray Tune
import ray
ray.shutdown()
ray.init(log_to_driver=True, ignore_reinit_error=True)

2021-08-01 14:25:17,655	INFO resource_spec.py:231 -- Starting Ray with 7.42 GiB memory available for workers and up to 3.73 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2021-08-01 14:25:18,798	INFO services.py:1193 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2021-08-01_14-25-17_653499_62/sockets/plasma_store',
 'raylet_ip_address': '172.28.0.2',
 'raylet_socket_name': '/tmp/ray/session_2021-08-01_14-25-17_653499_62/sockets/raylet',
 'redis_address': '172.28.0.2:6379',
 'session_dir': '/tmp/ray/session_2021-08-01_14-25-17_653499_62',
 'webui_url': 'localhost:8265'}

In [ ]:
config = {
        # These 3 configs below were defined earlier
        "model_name": model,
        # "task_name": task_name,
        # "data_dir": task_data_dir,
        "per_gpu_val_batch_size": 1,
        "per_gpu_train_batch_size": tune.choice([1]),
        "learning_rate": tune.uniform(1e-5, 5e-5),
        "weight_decay": tune.uniform(0.0, 0.3),
        "num_epochs": tune.choice([3, 5]),
        "max_steps": -1,  # We use num_epochs instead.
        "wandb": {
            "project": "pbt_transformers",
            "reinit": True,
            "allow_val_change": True
        }
    }

Now we can set up our Population Based Training scheduler

In [ ]:
from ray.tune.schedulers import PopulationBasedTraining

scheduler = PopulationBasedTraining(
        time_attr="training_iteration",
        metric="eval_acc",
        mode="max",
        perturbation_interval=2,
        hyperparam_mutations={
            "weight_decay": lambda: tune.uniform(0.0, 0.3).func(None),
            "learning_rate": lambda: tune.uniform(1e-5, 5e-5).func(None),
            "per_gpu_train_batch_size": [1],
        })

We also create a CLI reporter to view our results from the command line. We specify the hyperparameters we want to see from the command line, as well as what metrics we want to see. The metrics are the inputs to the tune.report we call we make in TuneTransformerTrainer.evaluate

In [ ]:
from ray.tune import CLIReporter

reporter = CLIReporter(
        parameter_columns={
            "weight_decay": "w_decay",
            "learning_rate": "lr",
            "per_gpu_train_batch_size": "train_bs/gpu",
            "num_epochs": "num_epochs"
        },
        metric_columns=[
            "eval_acc", "eval_loss", "epoch", "training_iteration"
        ])

In [ ]:
def train_transformer(config, checkpoint_dir=None):
  train_dataset, eval_dataset = tokenized_datasets["train"], tokenized_datasets["validation"]

  training_args = Seq2SeqTrainingArguments(
        output_dir=tune.get_trial_dir(),
        learning_rate=config["learning_rate"],
        do_train=True,
        do_eval=True,
        evaluation_strategy = "epoch",
        # Run eval after every epoch.
        eval_steps=(len(train_dataset) // config["per_gpu_train_batch_size"]) +
        1,
        # We explicitly set save to 0, and do checkpointing in evaluate instead
        save_steps=0,
        num_train_epochs=config["num_epochs"],
        max_steps=config["max_steps"],
        per_device_train_batch_size=config["per_gpu_train_batch_size"],
        per_device_eval_batch_size=config["per_gpu_val_batch_size"],
        warmup_steps=500,
        weight_decay=config["weight_decay"],
        logging_dir="./logs",
    )

Finally, we pass in our training function, config, PBT scheduler, and reporter to tune:

In [ ]:
analysis = tune.run(
        train_transformer,
        resources_per_trial={
            "cpu": 1,
            "gpu": 1
        },
        config=config,
        num_samples=3,
        scheduler=scheduler,
        keep_checkpoints_num=3,
        checkpoint_score_attr="training_iteration",
        progress_reporter=reporter,
        local_dir="./ray_results/",
        name="tune_transformer_pbt")

2021-08-01 14:25:20,987	WARNING worker.py:1134 -- The dashboard on node 1120301385f4 failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/ray/dashboard/dashboard.py", line 961, in <module>
    dashboard.run()
  File "/usr/local/lib/python3.7/dist-packages/ray/dashboard/dashboard.py", line 576, in run
    aiohttp.web.run_app(self.app, host=self.host, port=self.port)
  File "/usr/local/lib/python3.7/dist-packages/aiohttp/web.py", line 508, in run_app
    loop.run_until_complete(main_task)
  File "/usr/lib/python3.7/asyncio/base_events.py", line 587, in run_until_complete
    return future.result()
  File "/usr/local/lib/python3.7/dist-packages/aiohttp/web.py", line 411, in _run_app
    await site.start()
  File "/usr/local/lib/python3.7/dist-packages/aiohttp/web_runner.py", line 128, in start
    reuse_port=self._reuse_port,
  File "/usr/lib/python3.7/asyncio/base_events.py", line 1389, in create_server
    % (sa, err.strerror

== Status ==
Memory usage on this node: 7.2/12.7 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 1/2 CPUs, 1/1 GPUs, 0.0/7.42 GiB heap, 0.0/2.54 GiB objects (0/1.0 GPUType:V100)
Result logdir: /content/ray_results/tune_transformer_pbt
Number of trials: 3 (2 PENDING, 1 RUNNING)
+-------------------------------+----------+-------+-----------+-------------+----------------+--------------+
| Trial name                    | status   | loc   |   w_decay |          lr |   train_bs/gpu |   num_epochs |
|-------------------------------+----------+-------+-----------+-------------+----------------+--------------|
| train_transformer_4d655_00000 | RUNNING  |       | 0.285214  | 2.49816e-05 |              1 |            5 |
| train_transformer_4d655_00001 | PENDING  |       | 0.179598  | 3.92798e-05 |              1 |            5 |
| train_transformer_4d655_00002 | PENDING  |       | 0.0467984 | 1.62407e-05 |              1 |            3 |
+---------------------------

2021-08-01 14:26:01,700	WARNING util.py:137 -- The `experiment_checkpoint` operation took 17.65482997894287 seconds to complete, which may be a performance bottleneck.


== Status ==
Memory usage on this node: 7.5/12.7 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.42 GiB heap, 0.0/2.54 GiB objects (0/1.0 GPUType:V100)
Result logdir: /content/ray_results/tune_transformer_pbt
Number of trials: 3 (2 PENDING, 1 TERMINATED)
+-------------------------------+------------+-------+-----------+-------------+----------------+--------------+
| Trial name                    | status     | loc   |   w_decay |          lr |   train_bs/gpu |   num_epochs |
|-------------------------------+------------+-------+-----------+-------------+----------------+--------------|
| train_transformer_4d655_00000 | TERMINATED |       | 0.285214  | 2.49816e-05 |              1 |            5 |
| train_transformer_4d655_00001 | PENDING    |       | 0.179598  | 3.92798e-05 |              1 |            5 |
| train_transformer_4d655_00002 | PENDING    |       | 0.0467984 | 1.62407e-05 |              1 |            3 |
+------------

2021-08-01 14:26:03,878	WARNING util.py:137 -- The `start_trial` operation took 2.1643381118774414 seconds to complete, which may be a performance bottleneck.
(pid=806) 2021-08-01 14:26:07.274533: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-08-01 14:26:22,847	WARNING util.py:137 -- The `experiment_checkpoint` operation took 18.96555185317993 seconds to complete, which may be a performance bottleneck.
2021-08-01 14:26:22,856	WARNING ray_trial_executor.py:414 -- Over the last 60 seconds, the Tune event loop has been backlogged processing new results. Consider increasing your period of result reporting to improve performance.


...

== Status ==
Memory usage on this node: 7.7/12.7 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.42 GiB heap, 0.0/2.54 GiB objects (0/1.0 GPUType:V100)
Result logdir: /content/ray_results/tune_transformer_pbt
Number of trials: 3 (1 PENDING, 2 TERMINATED)
+-------------------------------+------------+-------+-----------+-------------+----------------+--------------+
| Trial name                    | status     | loc   |   w_decay |          lr |   train_bs/gpu |   num_epochs |
|-------------------------------+------------+-------+-----------+-------------+----------------+--------------|
| train_transformer_4d655_00000 | TERMINATED |       | 0.285214  | 2.49816e-05 |              1 |            5 |
| train_transformer_4d655_00001 | TERMINATED |       | 0.179598  | 3.92798e-05 |              1 |            5 |
| train_transformer_4d655_00002 | PENDING    |       | 0.0467984 | 1.62407e-05 |              1 |            3 |
+------------

2021-08-01 14:26:42,405	WARNING util.py:137 -- The `start_trial` operation took 2.411698818206787 seconds to complete, which may be a performance bottleneck.
(pid=920) 2021-08-01 14:26:46.748371: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-08-01 14:27:01,900	WARNING util.py:137 -- The `experiment_checkpoint` operation took 19.49385952949524 seconds to complete, which may be a performance bottleneck.


== Status ==
Memory usage on this node: 8.0/12.7 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 1/2 CPUs, 1/1 GPUs, 0.0/7.42 GiB heap, 0.0/2.54 GiB objects (0/1.0 GPUType:V100)
Result logdir: /content/ray_results/tune_transformer_pbt
Number of trials: 3 (1 RUNNING, 2 TERMINATED)
+-------------------------------+------------+-------+-----------+-------------+----------------+--------------+
| Trial name                    | status     | loc   |   w_decay |          lr |   train_bs/gpu |   num_epochs |
|-------------------------------+------------+-------+-----------+-------------+----------------+--------------|
| train_transformer_4d655_00000 | TERMINATED |       | 0.285214  | 2.49816e-05 |              1 |            5 |
| train_transformer_4d655_00001 | TERMINATED |       | 0.179598  | 3.92798e-05 |              1 |            5 |
| train_transformer_4d655_00002 | RUNNING    |       | 0.0467984 | 1.62407e-05 |              1 |            3 |
+------------

2021-08-01 14:27:18,990	WARNING util.py:137 -- The `experiment_checkpoint` operation took 16.840167999267578 seconds to complete, which may be a performance bottleneck.


== Status ==
Memory usage on this node: 7.7/12.7 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.42 GiB heap, 0.0/2.54 GiB objects (0/1.0 GPUType:V100)
Result logdir: /content/ray_results/tune_transformer_pbt
Number of trials: 3 (3 TERMINATED)
+-------------------------------+------------+-------+-----------+-------------+----------------+--------------+
| Trial name                    | status     | loc   |   w_decay |          lr |   train_bs/gpu |   num_epochs |
|-------------------------------+------------+-------+-----------+-------------+----------------+--------------|
| train_transformer_4d655_00000 | TERMINATED |       | 0.285214  | 2.49816e-05 |              1 |            5 |
| train_transformer_4d655_00001 | TERMINATED |       | 0.179598  | 3.92798e-05 |              1 |            5 |
| train_transformer_4d655_00002 | TERMINATED |       | 0.0467984 | 1.62407e-05 |              1 |            3 |
+-----------------------

We can now finetune our model by just calling the `train` method:

In [ ]:
batch_size = 1
args = Seq2SeqTrainingArguments(
    "test-summarization",
    evaluation_strategy = "epoch",
    warmup_steps=500,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.015,
    save_total_limit=1,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    report_to="wandb",
    adafactor = True, # recommended to use adafactor instead of adamw for T5
    learning_rate = 1e-4
)

PyTorch: setting up devices


In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Using amp fp16 backend


In [ ]:
# Clear space 
torch.cuda.empty_cache()

In [ ]:
# "model_T5" is saved in wandb.run.dir & will be uploaded at the end of training
trainer.save_model(os.path.join(wandb.run.dir, "model.T5"))

Saving model checkpoint to /content/wandb/run-20210801_142511-11cot1ae/files/model.T5
Configuration saved in /content/wandb/run-20210801_142511-11cot1ae/files/model.T5/config.json
Model weights saved in /content/wandb/run-20210801_142511-11cot1ae/files/model.T5/pytorch_model.bin
tokenizer config file saved in /content/wandb/run-20210801_142511-11cot1ae/files/model.T5/tokenizer_config.json
Special tokens file saved in /content/wandb/run-20210801_142511-11cot1ae/files/model.T5/special_tokens_map.json


In [ ]:
# Train the Model
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Original Text, __index_level_0__, Summary.
***** Running training *****
  Num examples = 607
  Num Epochs = 5
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 3035
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1310: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.831500,2.679664,31.420700,12.378000,27.145000,27.150100,12.538500
2,2.888400,2.426994,37.327100,20.596400,34.618200,34.742300,12.576900
3,2.420600,2.363035,38.050700,21.517900,35.418500,35.548400,12.129800
4,2.346900,2.348763,37.727700,21.024800,35.104700,35.234300,12.408700
5,2.068200,2.340661,38.363600,21.719400,35.559400,35.671600,12.375000


Saving model checkpoint to test-summarization/checkpoint-500
Configuration saved in test-summarization/checkpoint-500/config.json
Model weights saved in test-summarization/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-summarization/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-summarization/checkpoint-500/special_tokens_map.json
Deleting older checkpoint [test-summarization/checkpoint-3000] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Original Text, __index_level_0__, Summary.
***** Running Evaluation *****
  Num examples = 208
  Batch size = 1
Saving model checkpoint to test-summarization/checkpoint-1000
Configuration saved in test-summarization/checkpoint-1000/config.json
Model weights saved in test-summarization/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-summarization/checkpoint-1000

TrainOutput(global_step=3035, training_loss=2.6098846347563742, metrics={'train_runtime': 487.8748, 'train_samples_per_second': 6.221, 'train_steps_per_second': 6.221, 'total_flos': 48823497523200.0, 'train_loss': 2.6098846347563742, 'epoch': 5.0})

In [ ]:
# Save a model file from the current directory
wandb.save('trainer.T5')

[]

In [ ]:
# Save the model
wandb.run.save()

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [ ]:
# Finish WADNB run
wandb.finish()

loss,0.2
epoch,1
_runtime,660
_timestamp,1627828571
_step,12
train/loss,2.0682
train/learning_rate,0.0
train/epoch,5.0
train/global_step,3035
eval/loss,2.34066
eval/rouge1,38.3636


loss,▁
epoch,▁
_runtime,▁▃▄▄▅▅▆▆▇▇███
_timestamp,▁▃▄▄▅▅▆▆▇▇███
_step,▁▂▂▃▃▄▅▅▆▆▇▇█
train/loss,█▄▂▂▁▁
train/learning_rate,█▇▅▄▂▁
train/epoch,▁▁▂▃▄▅▅▆▇███
train/global_step,▁▁▂▃▄▅▅▆▇███
eval/loss,█▃▁▁▁
eval/rouge1,▁▇█▇█


Test on the test data

In [ ]:
# See what's in the test data
tokenized_datasets["test"]

Dataset({
    features: ['Original Text', 'Summary', '__index_level_0__', 'attention_mask', 'input_ids', 'labels'],
    num_rows: 53
})

In [ ]:
# Make predictions
predictions = trainer.predict(tokenized_datasets["test"])

The following columns in the test set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Original Text, __index_level_0__, Summary.
***** Running Prediction *****
  Num examples = 53
  Batch size = 1


In [ ]:
# Create a function to decode predictions
def translate_predictions(prediction):
  for i in prediction:
    decoded_prediction = tokenizer.batch_decode(i, skip_special_tokens=True)
    return decoded_prediction

In [ ]:
# Decode predictions
predictions = pd.DataFrame(predictions)
decoded_predictions = predictions.apply(translate_predictions)

/usr/local/lib/python3.7/dist-packages/pandas/core/internals/construction.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  values = np.array([convert(v) for v in values])


In [ ]:
# Maximise column width
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [ ]:
# Take a random sample of the dataset
import random
random.seed(12)
randomlist = []
for i in range(0,20):
  n = random.randint(1,len(tokenized_datasets["test"]))
  randomlist.append(n)
print(randomlist)

[31, 18, 43, 34, 43, 23, 10, 25, 1, 24, 31, 18, 42, 52, 30, 45, 39, 15, 36, 1]


In [ ]:
# See the corresponding decoded predictions
samples = pd.concat([decoded_predictions.iloc[i] for i in randomlist], axis = 1).T

In [ ]:
# Find the originals 
originals = pd.concat([tac_test.iloc[i] for i in randomlist], axis = 1).T

In [ ]:
# Reset sample index
samples.reset_index(drop=True, inplace=True)

In [ ]:
# Reset originals index
originals.reset_index(drop=True, inplace=True)

In [ ]:
# Combine the datasets to make a comparisons dataframe
comparisons = pd.concat([samples, originals], axis = 1)

In [ ]:
# Label and look at dataframe
comparisons.set_axis(['T5 Prediction', 'Summary', 'Original Text'], axis=1, inplace=True)
comparisons

,T5 Prediction,Summary,Original Text
0,the court of law governing the terms is in location russian federation russia,the court of law governing the terms is in location st petersburg russia,in these terms and other special documents the vk site administration hereinafter the site administration administration is understood as llc v kontakte a legal entity created under the laws of the russian federation and registered at prem 1 n bld 12 14 lit a khersonskaya st st petersburg russia 191024
1,mewe reserves the right to remove objectionable content without notice,your content can be deleted if you violate the terms,mewe reserves the right to remove objectionable content without notice mewe can remove any content or information you post at mewe if we believe that it violates our terms of service
2,the service does not use the brainly services,users agree not to submit libelous harassing or threatening content,not use the brainly services do anything unlawful misleading malicious or discriminatory
3,this service uses targeted cookies for advertising,this service tracks you on other websites,we also use retargeting cookies to present you with patreon advertising on other websites
4,the service does not use the brainly services,users agree not to submit libelous harassing or threatening content,not use the brainly services do anything unlawful misleading malicious or discriminatory
5,the service is committed to the privacy of users,this service respects your browsers do not track dnt headers,one concrete way we commit to user privacy is by honoring do not track “dnt” browser settings there’s no consensus on how best to do this but we have adopted an approach that we believe honors the fundamental pro privacy aims of the dnt standard
6,you can use googles privacy policy for your personal data,users are subject to googles privacy policy,by using or visiting the youtube website or any youtube products software data feeds and services provided to you on from or through the youtube website collectively the service you signify your agreement to 1 these terms and conditions the terms of service 2 googles privacy policy found at and incorporated herein by reference and 3 youtubes community guidelines found at and also incorporated herein by reference
7,the service does not promote or condone any content,discogs does not condone any ideas contained in the items listed via the service,e we do not promote or condone any ideas or messages contained in the user generated content available through the service
8,any liability on behalf of the service is only limited to 1 000,any liability on behalf of the service is only limited to 1 000,and ii for any damages losses and or causes of action exceeding one thousand u s dollars us 1 000 in the aggregate
9,no logs are kept at any part,user logs are never stored at any component of infrastructure,we will never keep logs at any component of our infrastructure


In [ ]:
# Convert to csv
decoded_predictions.to_csv("decoded_predictions_T5_nodups.csv")

In [ ]:
# Download predictions made
from google.colab import files
files.download("decoded_predictions_T5_nodups.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Comparisons dataframe to csv
comparisons.to_csv("comparisons_T5_nodups.csv")

In [ ]:
# Download the file
files.download("comparisons_T5_nodups.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>